Read Water quality parameters from shapefile
List all satellite dates 
find nearest satellite image for each measurement
associate satellite bands for each measurement
train models 
evaluate results 


In [1]:
satellite="Landsat" 

In [3]:
#mport xarray as xr
import numpy as np
import pandas as pd
import os
import geopandas as gpd
#initialize variables 
DATA_DIR='DATA/vrana/Landsat_new'
file_bands='DATA/bands_landsat.csv'
band_names = ["B01","B02","B03","B04","B05","B06","B07", "B10"]


#define correct variables for each satellite 
if satellite== 'Landsat':
    DATA_DIR='DATA/vrana/Landsat_new'
    file_bands='DATA/bands_landsat.csv'

 
    band_names = ["B01","B02","B03","B04","B05","B06","B07", "B10"]
if satellite=='Sentinel_2':
    DATA_DIR='DATA/vrana/Sentinel_new'
    file_bands='DATA/bands_sentinel.csv'

    band_names = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
  
if satellite=='Planet_Scope':
    DATA_DIR='DATA/vrana/transfer.pcloud.PnXOdLhM'
    band_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8' ]
    file_bands='DATA/bands_planet.csv'

 
print(satellite)
in_sutu_m = gpd.read_file('DATA/vrana/in_situ_230_standardize_32633.shp')


Landsat


In [ ]:
import os
import re
from datetime import datetime
import pandas as pd
import geopandas as gpd
import rasterio

base_path = DATA_DIR
base_path = DATA_DIR
if os.path.exists(file_bands):
    print("File exists!")
    df=pd.read_csv(file_bands)
else:
    print("File does not exist.") 
    # --- Step 1: Extract folder dates and map to folder paths ---
    folder_date_map = {}
    for f in os.listdir(base_path):
        full_path = os.path.join(base_path, f)
        if os.path.isdir(full_path):
            match = re.search(r"(\d{8})", f)
            if match:
                date_obj = datetime.strptime(match.group(1), "%Y%m%d")
                folder_date_map[date_obj] = full_path

    folder_dates = sorted(folder_date_map.keys())

    # --- Step 2: Example gdf with geometry + date column ---
    # gdf = gpd.read_file("points.geojson")
    # gdf['date'] = pd.to_datetime(gdf['date'])
    gdf=in_sutu_m.copy()
    gdf['datum_Sat']=gdf['date']
    # --- Helper to find nearest available folder date ---
    def find_nearest_date(target_date, date_list):
        return min(date_list, key=lambda d: abs(d - target_date))

    # --- Step 3: Extract pixel values into separate band columns ---
    
    # Prepare empty columns in gdf
    for b in band_names:
        gdf[b] = None

    for idx, row in gdf.iterrows():
        nearest_date = find_nearest_date(row['date'], folder_dates)
        folder_path = folder_date_map[nearest_date]
        print(row['date'],nearest_date,folder_path)

        point = row.geometry

        # Read TIFFs from folder
        tiff_files = sorted([f for f in os.listdir(folder_path) if (f.lower().endswith('.tiff') or f.lower().endswith('.tif'))])

        for tiff_file in tiff_files:
            # Extract band code from filename (e.g., 'B04')
            if satellite=="Planet_Scope":
                    match_band = re.search(r'B(\d{1,2}[A]?)', tiff_file)
            else:
                    match_band = re.search(r'_B(\d+[A]?)_', tiff_file)
            if match_band:
                band_code = "B" + match_band.group(1)

                if band_code in band_names:
                    with rasterio.open(os.path.join(folder_path, tiff_file)) as src:
                        # Ensure CRS matches
                        if gdf.crs != src.crs:
                            point = gdf.to_crs(src.crs).iloc[idx].geometry
                        val = list(src.sample([(point.x, point.y)]))[0][0]
                        gdf.at[idx, band_code] = val
                        gdf.at[idx, 'datum_Sat'] = nearest_date
                    
                        print(idx,row['date'],band_code,val,nearest_date )
    df=gdf.dropna().copy()
    df.to_csv(file_bands)
